# Sentiment Analysis with an RNN

In this notebook, you'll implement a recurrent neural network that performs sentiment analysis. 
>Using an RNN rather than a strictly feedforward network is more accurate since we can include information about the *sequence* of words. 

Here we'll use a dataset of movie reviews, accompanied by sentiment labels: positive or negative.

<img src="assets/reviews_ex.png" width=40%>

### Network Architecture

The architecture for this network is shown below.

<img src="assets/network_diagram.png" width=40%>

>**First, we'll pass in words to an embedding layer.** We need an embedding layer because we have tens of thousands of words, so we'll need a more efficient representation for our input data than one-hot encoded vectors. You should have seen this before from the Word2Vec lesson. You can actually train an embedding with the Skip-gram Word2Vec model and use those embeddings as input, here. However, it's good enough to just have an embedding layer and let the network learn a different embedding table on its own. *In this case, the embedding layer is for dimensionality reduction, rather than for learning semantic representations.*

>**After input words are passed to an embedding layer, the new embeddings will be passed to LSTM cells.** The LSTM cells will add *recurrent* connections to the network and give us the ability to include information about the *sequence* of words in the movie review data. 

>**Finally, the LSTM outputs will go to a sigmoid output layer.** We're using a sigmoid function because positive and negative = 1 and 0, respectively, and a sigmoid will output predicted, sentiment values between 0-1. 

We don't care about the sigmoid outputs except for the **very last one**; we can ignore the rest. We'll calculate the loss by comparing the output at the last time step and the training label (pos or neg).

---
### Load in and visualize the data

In [201]:
import numpy as np

# read data from text files
with open('data/tweets_full.txt', 'r') as f:
    reviews = f.read()
with open('data/tweets_full_target.txt', 'r') as f:
    labels = f.read()

In [202]:
# read data from text files
with open('data/tweets_authors.txt', 'r') as f:
    authors = f.read()

In [203]:
print(reviews[:1000])
print()
print(labels[:20])
print()
print(authors[:20])

@tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =[ 
Layin n bed with a headache  ughhhh...waitin on your call... 
Funeral ceremony...gloomy friday... 
wants to hang out with friends SOON! 
@dannycastillo We want to trade with someone who has Houston tickets, but no one will. 
Re-pinging @ghostridah14: why didn't you go to prom? BC my bf didn't like my friends 
I should be sleep, but im not! thinking about an old friend who I want. but he's married now. damn, &amp; he wants me 2! scandalous! 
Hmmm. http://www.djhero.com/ is down 
@charviray Charlene my love. I miss you 
@kelcouch I'm sorry  at least it's Friday? 
cant fall asleep 
Choked on her retainers 
Ugh! I have to beat this stupid song to get to the next  rude! 
@BrodyJenner if u watch the hills in london u will realise what tourture it is because were weeks and weeks late  i just watch itonlinelol 
Got the news 
The storm is here and the electricity is gone 
@annarosekerr agreed 
So sleep

## Data pre-processing

The first step when building a neural network model is getting your data into the proper form to feed into the network. Since we're using embedding layers, we'll need to encode each word with an integer. We'll also want to clean it up a bit.

You can see an example of the reviews data above. Here are the processing steps, we'll want to take:
>* We'll want to get rid of periods and extraneous punctuation.
* Also, you might notice that the reviews are delimited with newline characters `\n`. To deal with those, I'm going to split the text into each review using `\n` as the delimiter. 
* Then I can combined all the reviews back together into one big string.

First, let's remove all punctuation. Then get all the text without the newlines and split it into individual words.

In [204]:
from string import punctuation

# get rid of punctuation
reviews = reviews.lower() # lowercase, standardize
all_text = ''.join([c for c in reviews if c not in punctuation])

# split by new lines and spaces
reviews_split = all_text.split('\n')
all_text = ' '.join(reviews_split)

# create a list of words
words = all_text.split()

In [205]:
words[:39]

['tiffanylue',
 'i',
 'know',
 'i',
 'was',
 'listenin',
 'to',
 'bad',
 'habit',
 'earlier',
 'and',
 'i',
 'started',
 'freakin',
 'at',
 'his',
 'part',
 'layin',
 'n',
 'bed',
 'with',
 'a',
 'headache',
 'ughhhhwaitin',
 'on',
 'your',
 'call',
 'funeral',
 'ceremonygloomy',
 'friday',
 'wants',
 'to',
 'hang',
 'out',
 'with',
 'friends',
 'soon',
 'dannycastillo',
 'we']

### Encoding the words

The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our reviews into integers so they can be passed into the network.

> **Exercise:** Now you're going to encode the words with integers. Build a dictionary that maps words to integers. Later we're going to pad our input vectors with zeros, so make sure the integers **start at 1, not 0**.
> Also, convert the reviews to integers and store the reviews in a new list called `reviews_ints`. 

In [206]:
# feel free to use this import 
from collections import Counter

## Build a dictionary that maps words to integers
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = []
for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])
    
reviews_ints = reviews_ints[:-1] # Remove the empty line at the end

**Test your code**

As a text that you've implemented the dictionary correctly, print out the number of unique words in your vocabulary and the contents of the first, tokenized review.

In [207]:
# stats about vocabulary
print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()

# print tokens in first review
print('Tokenized review: \n', reviews_ints[-1])

Unique words:  53747

Tokenized review: 
 [14919, 14884, 1038, 50, 2845, 3, 2598, 6, 1, 16, 99, 1954, 4206, 309, 950, 53745, 53746, 53747]


### Encoding the labels

Our labels are "positive" or "negative". To use these labels in our network, we need to convert them to 0 and 1.


In [208]:
# 1=positive, 0=negative label conversion
labels_split = labels.split('\n')
labels_split = [label.strip() for label in labels_split]

label_names = ['neutral', 'worry', 'happiness', 'sadness', 'love', 'surprise',
              'fun', 'relief', 'hate', 'empty', 
             'enthusiasm', 'boredom', 'anger']

label_to_int = {k:i for i, k in enumerate(label_names) }
int_to_label = {v: k for k, v in label_to_int.items()}
encoded_labels = np.array([label_to_int[label] for label in labels_split[:-1]])



In [209]:
print(label_to_int)

print("Len of tweets = {}".format(len(reviews_ints)))
print("Len of labels = {}".format(len(encoded_labels)))

{'neutral': 0, 'worry': 1, 'happiness': 2, 'sadness': 3, 'love': 4, 'surprise': 5, 'fun': 6, 'relief': 7, 'hate': 8, 'empty': 9, 'enthusiasm': 10, 'boredom': 11, 'anger': 12}
Len of tweets = 40000
Len of labels = 40000


### Encode the authors of each tweet

**Remove punctuation**

In [210]:
# get rid of punctuation
authors = authors.lower() # lowercase, standardize
all_authors_text = ''.join([c for c in authors if c not in punctuation])
    
# split by new lines and spaces
authors_split = all_authors_text.split('\n')
all_authors_text = ' '.join(authors_split)

# create a list of words
all_authors = all_authors_text.split()

print(authors_split[:5])

['xoshayzers ', 'wannamama ', 'coolfunky ', 'czareaquino ', 'xkilljoyx ']


**Encoding the authors**

In [211]:
# feel free to use this import 
from collections import Counter

## Build a dictionary that maps words to integers
counts_auth = Counter(all_authors)
vocab_auth = sorted(counts_auth, key=counts_auth.get, reverse=True)
vocab_to_int_auth = {word: ii for ii, word in enumerate(vocab_auth, 1)}

# Add element for non-existing authors
vocab_to_int_auth["None"] = len(vocab_to_int_auth)+1

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
authors_ints = [[vocab_to_int_auth[word] for word in all_authors]]

authors_ints = np.array(authors_ints).squeeze()

print(authors_ints)

[ 1065  3901  3902 ... 33861 33862   244]


### Removing Outliers

As an additional pre-processing step, we want to make sure that our reviews are in good shape for standard processing. That is, our network will expect a standard input text size, and so, we'll want to shape our reviews into a specific length. We'll approach this task in two main steps:

1. Getting rid of extremely long or short reviews; the outliers
2. Padding/truncating the remaining data so that we have reviews of the same length.

Before we pad our review text, we should check for reviews of extremely short or long lengths; outliers that may mess with our training.

In [212]:
# outlier review stats
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 0
Maximum review length: 33


Okay, a couple issues here. We seem to have one review with zero length. And, the maximum review length is way too many steps for our RNN. We'll have to remove any super short reviews and truncate super long reviews. This removes outliers and should allow our model to train more efficiently.

> **Exercise:** First, remove *any* reviews with zero length from the `reviews_ints` list and their corresponding label in `encoded_labels`.

In [213]:
print('Number of reviews before removing outliers: ', len(reviews_ints))

## remove any reviews/labels with zero length from the reviews_ints list.

# get indices of any reviews with length 0
non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review) != 0]

# remove 0-length reviews and their labels
reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print('Number of reviews after removing outliers: ', len(reviews_ints))

Number of reviews before removing outliers:  40000
Number of reviews after removing outliers:  40000


---
## Padding sequences

To deal with both short and very long reviews, we'll pad or truncate all our reviews to a specific length. For reviews shorter than some `seq_length`, we'll pad with 0s. For reviews longer than `seq_length`, we can truncate them to the first `seq_length` words. A good `seq_length`, in this case, is 200.

> **Exercise:** Define a function that returns an array `features` that contains the padded data, of a standard size, that we'll pass to the network. 
* The data should come from `review_ints`, since we want to feed integers to the network. 
* Each row should be `seq_length` elements long. 
* For reviews shorter than `seq_length` words, **left pad** with 0s. That is, if the review is `['best', 'movie', 'ever']`, `[117, 18, 128]` as integers, the row will look like `[0, 0, 0, ..., 0, 117, 18, 128]`. 
* For reviews longer than `seq_length`, use only the first `seq_length` words as the feature vector.

As a small example, if the `seq_length=10` and an input review is: 
```
[117, 18, 128]
```
The resultant, padded sequence should be: 

```
[0, 0, 0, 0, 0, 0, 0, 117, 18, 128]
```

**Your final `features` array should be a 2D array, with as many rows as there are reviews, and as many columns as the specified `seq_length`.**

This isn't trivial and there are a bunch of ways to do this. But, if you're going to be building your own deep learning networks, you're going to have to get used to preparing your data.

In [214]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

In [215]:
# Test your implementation!

seq_length = 25

features = pad_features(reviews_ints, seq_length=seq_length)

## test statements - do not change - ##
assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches 
print(features[:30,:12])

[[    0     0     0     0     0     0     0     0 14939     1    54     1]
 [    0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0 14942    61]
 [    0     0     0     0     0     0     0     0     0     0  7172 14943]
 [    1   134    23   131    19    13    27   362    60    85   222   242]
 [    0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0   281]
 [ 9423    70    57   207

## Training, Validation, Test

With our data in nice shape, we'll split it into training, validation, and test sets.

> **Exercise:** Create the training, validation, and test sets. 
* You'll need to create sets for the features and the labels, `train_x` and `train_y`, for example. 
* Define a split fraction, `split_frac` as the fraction of data to **keep** in the training set. Usually this is set to 0.8 or 0.9. 
* Whatever data is left will be split in half to create the validation and *testing* data.

In [216]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))


			Feature Shapes:
Train set: 		(32000, 25) 
Validation set: 	(4000, 25) 
Test set: 		(4000, 25)


**Split the authors accordingly**

In [217]:
authors_x, remaining_authors = authors_ints[:split_idx], authors_ints[split_idx:]

authors_val, authors_test = remaining_authors[:test_idx], remaining_authors[test_idx:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(authors_x.shape), 
      "\nValidation set: \t{}".format(authors_val.shape),
      "\nTest set: \t\t{}".format(authors_test.shape))

			Feature Shapes:
Train set: 		(32000,) 
Validation set: 	(4000,) 
Test set: 		(4000,)


**Check your work**

With train, validation, and test fractions equal to 0.8, 0.1, 0.1, respectively, the final, feature data shapes should look like:
```
                    Feature Shapes:
Train set: 		 (train_size, word_length) 
Validation set: 	(val_size, word_length) 
Test set: 		  (test_size, word_length)
```

---
## DataLoaders and Batching

After creating training, test, and validation data, we can create DataLoaders for this data by following two steps:
1. Create a known format for accessing our data, using [TensorDataset](https://pytorch.org/docs/stable/data.html#) which takes in an input set of data and a target set of data with the same first dimension, and creates a dataset.
2. Create DataLoaders and batch our training, validation, and test Tensor datasets.

```
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
train_loader = DataLoader(train_data, batch_size=batch_size)
```

This is an alternative to creating a generator function for batching our data into full batches.

In [218]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y), torch.from_numpy(authors_x))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y), torch.from_numpy(authors_val))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y), torch.from_numpy(authors_test))

# dataloaders
batch_size = 50

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [219]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y, author = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)
print()
print('Sample label size: ', author.size()) # batch_size
print('Sample label: \n', author)

Sample input size:  torch.Size([50, 25])
Sample input: 
 tensor([[    0,     0,     0,  ...,  9237,  4906,  1294],
        [    0,  3921,    74,  ...,   511,     8,   225],
        [    0,     0,     0,  ...,     6,    23,    49],
        ...,
        [13373,     1,    74,  ...,  1105,    18,  4339],
        [    0,     0, 24141,  ...,    14,     8,   198],
        [    0,     0, 39963,  ...,     3,  3030, 12620]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([ 4,  0,  2,  6,  3,  1,  0,  6,  0,  9,  3,  4,  2,  4,  5,  5,  3,  2,
         4,  1,  0,  3,  1,  3,  8,  0,  5,  1,  0,  5,  1,  3,  3,  4,  9,  1,
         0,  3,  6, 10,  1,  0,  3, 10,  4,  5,  1,  0,  0,  1])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([  956, 11869, 27600, 27333,  5819,  4839,   649, 23497, 10498, 19137,
         1938,  1632, 20676,   329, 24256,   566,  6779,     1, 20215,    92,
         8774, 15186, 15444, 15162, 18392, 18776,  5781,     3, 21114,  5592,
        18086,

---
# Sentiment Network with PyTorch

Below is where you'll define the network.

<img src="assets/network_diagram.png" width=40%>

The layers are as follows:
1. An [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) that converts our word tokens (integers) into embeddings of a specific size.
2. An [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) defined by a hidden_state size and number of layers
3. A fully-connected output layer that maps the LSTM layer outputs to a desired output_size
4. A sigmoid activation layer which turns all outputs into a value 0-1; return **only the last sigmoid output** as the output of this network.

### The Embedding Layer

We need to add an [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) because there are 74000+ words in our vocabulary. It is massively inefficient to one-hot encode that many classes. So, instead of one-hot encoding, we can have an embedding layer and use that layer as a lookup table. You could train an embedding layer using Word2Vec, then load it here. But, it's fine to just make a new layer, using it for only dimensionality reduction, and let the network learn the weights.


### The LSTM Layer(s)

We'll create an [LSTM](https://pytorch.org/docs/stable/nn.html#lstm) to use in our recurrent network, which takes in an input_size, a hidden_dim, a number of layers, a dropout probability (for dropout between multiple layers), and a batch_first parameter.

Most of the time, you're network will have better performance with more layers; between 2-3. Adding more layers allows the network to learn really complex relationships. 

> **Exercise:** Complete the `__init__`, `forward`, and `init_hidden` functions for the SentimentRNN model class.

Note: `init_hidden` should initialize the hidden and cell state of an lstm layer to all zeros, and move those state to GPU, if available.

In [220]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [221]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding_author = nn.Embedding(len(vocab_to_int_auth)+1, hidden_dim) # make it same dimension as h_t
            

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.7)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        

    def forward(self, x, hidden, author):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        # embeddings and lstm_out
        batch_size = x.size(0)
        
        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        author_embedded = self.embedding_author
       
        lstm_out, hidden = self.lstm(embeds, hidden)
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
       
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        # sigmoid function
        softmax_out = self.softmax(out)
        # reshape to be batch_size first
        softmax_out = softmax_out.view(batch_size, seq_length, -1)
        softmax_out = softmax_out[:, -1] # get last batch of labels

        # return last sigmoid output and hidden state
        return softmax_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

## Save and Load models

In [222]:
# Saving function
def save_model (model):
    #other information about the model
    torch.save(model.state_dict(), 'data/lstm_best.pth')

## Instantiate the network

Here, we'll instantiate the network. First up, defining the hyperparameters.

* `vocab_size`: Size of our vocabulary or the range of values for our input, word tokens.
* `output_size`: Size of our desired output; the number of class scores we want to output (pos/neg).
* `embedding_dim`: Number of columns in the embedding lookup table; size of our embeddings.
* `hidden_dim`: Number of units in the hidden layers of our LSTM cells. Usually larger is better performance wise. Common values are 128, 256, 512, etc.
* `n_layers`: Number of LSTM layers in the network. Typically between 1-3

> **Exercise:** Define the model  hyperparameters.


In [223]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 13
embedding_dim = 256
hidden_dim = 100
n_layers = 1

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(53748, 256)
  (embedding_author): Embedding(33864, 100)
  (lstm): LSTM(256, 100, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.7, inplace=False)
  (fc): Linear(in_features=100, out_features=13, bias=True)
  (softmax): LogSoftmax()
)


/home/pablo/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/nn/modules/rnn.py:47: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


---
## Training

Below is the typical training code. If you want to do this yourself, feel free to delete all this code and implement it yourself. You can also add code to save a model by name.

>We'll also be using a new kind of cross entropy loss, which is designed to work with a single Sigmoid output. [BCELoss](https://pytorch.org/docs/stable/nn.html#bceloss), or **Binary Cross Entropy Loss**, applies cross entropy loss to a single value between 0 and 1.

We also have some data and training hyparameters:

* `lr`: Learning rate for our optimizer.
* `epochs`: Number of times to iterate through the training dataset.
* `clip`: The maximum gradient value to clip at (to prevent exploding gradients).

In [224]:
# loss and optimization functions
lr=0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(net.parameters(), lr=lr)
# Adamax


In [225]:
def train(net):
    # training params

    epochs = 10 # 3-4 is approx where I noticed the validation loss stop decreasing

    counter = 0
    print_every = 100
    clip=5 # gradient clipping
    best_loss = np.inf
    
    # move model to GPU, if available
    if(train_on_gpu):
        net.cuda()

    net.train()
    # train for some number of epochs
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)

        # batch loop
        for inputs, labels, authors in train_loader:
            counter += 1

            if(train_on_gpu):
                inputs, labels, authors = inputs.cuda(), labels.cuda(), authors.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h, authors)

            # calculate the loss and perform backprop
            loss = criterion(output.squeeze(), labels)
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            optimizer.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for inputs, labels, authors in valid_loader:

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    if(train_on_gpu):
                        inputs, labels, authors = inputs.cuda(), labels.cuda(), authors.cuda()

                    output, val_h = net(inputs, val_h, authors)
                    val_loss = criterion(output.squeeze(), labels)

                    val_losses.append(val_loss.item())

                net.train()
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.6f}...".format(loss.item()),
                      "Val Loss: {:.6f}".format(np.mean(val_losses)))

                if np.mean(val_losses) < best_loss:
                    print("Val loss improved to {:.6f}. Saving model...".format(np.mean(val_losses)))
                    best_loss = np.mean(val_losses)

                    save_model(net)
                    #net.load_state_dict(torch.load("data/lstm_best.pth"))
                    
    return net

best_net = train(net)

Epoch: 1/30... Step: 100... Loss: 2.121228... Val Loss: 2.260687
Val loss improved to 2.260687. Saving model...
Epoch: 1/30... Step: 200... Loss: 1.984893... Val Loss: 2.266338
Epoch: 1/30... Step: 300... Loss: 2.358399... Val Loss: 2.222852
Val loss improved to 2.222852. Saving model...
Epoch: 1/30... Step: 400... Loss: 1.985832... Val Loss: 2.219423
Val loss improved to 2.219423. Saving model...
Epoch: 1/30... Step: 500... Loss: 2.014025... Val Loss: 2.185413
Val loss improved to 2.185413. Saving model...
Epoch: 1/30... Step: 600... Loss: 1.913555... Val Loss: 2.176564
Val loss improved to 2.176564. Saving model...
Epoch: 2/30... Step: 700... Loss: 1.949304... Val Loss: 2.098589
Val loss improved to 2.098589. Saving model...
Epoch: 2/30... Step: 800... Loss: 2.055647... Val Loss: 2.065220
Val loss improved to 2.065220. Saving model...
Epoch: 2/30... Step: 900... Loss: 1.990028... Val Loss: 2.047651
Val loss improved to 2.047651. Saving model...
Epoch: 2/30... Step: 1000... Loss: 2.05

Epoch: 18/30... Step: 11500... Loss: 0.835622... Val Loss: 3.330767
Epoch: 19/30... Step: 11600... Loss: 0.499693... Val Loss: 3.396364
Epoch: 19/30... Step: 11700... Loss: 0.580205... Val Loss: 3.562249
Epoch: 19/30... Step: 11800... Loss: 0.714735... Val Loss: 3.505370
Epoch: 19/30... Step: 11900... Loss: 0.732818... Val Loss: 3.560295
Epoch: 19/30... Step: 12000... Loss: 0.454642... Val Loss: 3.569531
Epoch: 19/30... Step: 12100... Loss: 0.571471... Val Loss: 3.631360
Epoch: 20/30... Step: 12200... Loss: 0.389745... Val Loss: 3.682047
Epoch: 20/30... Step: 12300... Loss: 0.532012... Val Loss: 3.603214
Epoch: 20/30... Step: 12400... Loss: 0.625362... Val Loss: 3.659446
Epoch: 20/30... Step: 12500... Loss: 0.633000... Val Loss: 3.667255
Epoch: 20/30... Step: 12600... Loss: 0.734174... Val Loss: 3.693313
Epoch: 20/30... Step: 12700... Loss: 0.534035... Val Loss: 3.586832
Epoch: 20/30... Step: 12800... Loss: 0.639019... Val Loss: 3.612944
Epoch: 21/30... Step: 12900... Loss: 0.535047...

---
## Testing

There are a few ways to test your network.

* **Test data performance:** First, we'll see how our trained model performs on all of our defined test_data, above. We'll calculate the average loss and accuracy over the test data.

* **Inference on user-generated data:** Second, we'll see if we can input just one example review at a time (without a label), and see what the trained model predicts. Looking at new, user input data like this, and predicting an output label, is called **inference**.

In [226]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = best_net.init_hidden(batch_size)

best_net.eval()
# iterate over test data
for inputs, labels, authors in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels, authors = inputs.cuda(), labels.cuda(), authors.cuda()
    
    # get predicted outputs
    output, h = best_net(inputs, h, authors)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels)
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    predicted, idx = torch.topk(output, 1)

    # compare predictions to true label
    correct_tensor = idx.eq(labels.view_as(idx))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 4.587
Test accuracy: 0.251


### Inference on a test review

You can change this test_review to any text that you want. Read it and think: is it pos or neg? Then see if your model predicts correctly!
    
> **Exercise:** Write a `predict` function that takes in a trained net, a plain text_review, and a sequence length, and prints out a custom statement for a positive or negative review!
* You can use any functions that you've already defined or define any helper functions you want to complete `predict`, but it should just take in a trained net, a text review, and a sequence length.


In [227]:
# negative test review
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'


In [228]:
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])

    return test_ints

# test code and generate tokenized review
test_ints = tokenize_review(test_review_neg)
print(test_ints)

[[3, 805, 201, 1, 16, 409, 2217, 26, 1300, 6, 1, 76, 5, 358, 55, 30, 201, 64, 118, 2217, 6, 3, 43882, 26, 753]]


In [229]:
# test sequence padding
seq_length=35
features = pad_features(test_ints, seq_length)

print(features)

[[    0     0     0     0     0     0     0     0     0     0     3   805
    201     1    16   409  2217    26  1300     6     1    76     5   358
     55    30   201    64   118  2217     6     3 43882    26   753]]


In [230]:
# test conversion to tensor and pass into your model
feature_tensor = torch.from_numpy(features)
print(feature_tensor.size())

torch.Size([1, 35])


In [231]:
def predict(net, test_review, author, sequence_length=200):
    
    net.eval()
    
    # tokenize review
    test_ints = tokenize_review(test_review)
    
    try:
        test_auth = vocab_to_int_auth[author]
    except:
        test_auth = vocab_to_int_auth["None"]
        
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
        
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    author_tensor = torch.tensor(test_auth)

    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
        author_tensor = author_tensor.cuda()
    
    # get the output from the model
    output, h = net(feature_tensor, h, author_tensor)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    # printing output value, before rounding
    val, idx = torch.topk(output, 1)
    
    idx = idx.cpu().numpy()[0][0]

    print('Prediction: {} '.format(int_to_label[idx]))
    print("Prediction value: {:.6f}".format(val.detach().cpu().numpy()[0][0]))
        

In [232]:
# positive test review
test_review_pos = "hello there asshole"
author = "tiffanylue"

In [233]:
# call function
seq_length=35 # good to use the length that was trained on

predict(best_net, test_review_pos, author, seq_length)

Prediction: neutral 
Prediction value: -0.001026


### Try out test_reviews of your own!

Now that you have a trained model and a predict function, you can pass in _any_ kind of text and this model will predict whether the text has a positive or negative sentiment. Push this model to its limits and try to find what words it associates with positive or negative.

Later, you'll learn how to deploy a model like this to a production environment so that it can respond to any kind of user data put into a web app!